Code pour calculer la diversité des oeuvres par rapport aux périodes : l'indice de Shannon.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import entropy
import matplotlib.pyplot as plt

# Je reprends le début du même script que j'avais fait pour visualiser le nombre de manuscrits par période et par genre
categories = {
    'IV-VIIIe siècle': ['VIe siècle', 'VIIe siècle', 'VIIIe siècle', 'V-VIe siècle', 'VI-VIIe siècle', 'Ve-VIe siècle'],
    'IX-XIe siècle': ['IXe siècle', 'Xe siècle', 'XIe siècle', 'VIII-IXe siècle', 'X-XIe siècle', 'IX-Xe siècle', 'IX-XIe siècle', 'VIII-Xe siècle', 'VIII-XIe siècle'],
    'XII-XVIe siècle': ['XIIe siècle', 'XIIIe siècle', 'XIVe siècle', 'XVe siècle', 'XI-XVe  siècle', 'XI-XVe siècle', 'XII-XIIIe siècle', 'XIIIe-XIVe siècle', 'XIII-XVe siècle', 'XV-XVIe siècle', 'XIII-XIVe siècle', 'XIV-XVe siècle', 'XIIe-XIVe siècle', 'XI-XIIe siècle']
}

def assign_period(century):
    for period, centuries in categories.items():
        if century in centuries:
            return period
    return 'Non spécifié'

# Fonction qui me permet de calculer la diversité de Shannon : on calcule la fréquence d'apparition de chaque oeuvre dans notre jeu de données puis on lui applique  la fonction"entropy" de scipy.stats (c'est l'indice de Shannon)
def shannon_diversity(work_titles):
    value_counts = work_titles.value_counts()
    probabilities = value_counts / len(work_titles)
    return entropy(probabilities)


file_path = 'C:\\Users\\ADMIN\\Desktop\\Ecole_des_Chartes\\Memoire_2\\manuscripts_data_2\\manuscripts_data_vr\\wits_table4.xlsx'  # Remplace par le chemin de ton fichier Excel
df = pd.read_excel(file_path)

# Je filtre et garde seulement les colonnes qui me seront utiles
df_filtered = df[['witSignature', 'Century_witOrigDate', 'workTitle']]

# Je créé une nouvelle colonne "Wits_sample" qui attribue à chaque manuscrit sa période.
df_filtered['Wits_sample'] = df_filtered['Century_witOrigDate'].apply(assign_period)

# J'enlève de mon étude les manuscrits dont la date n'est pas spécifié : j'ai conscience que cela représentera un biais pour mes données
df_filtered = df_filtered[df_filtered['Wits_sample'] != 'Non spécifié']

# J'applique ma fonction pour chaque période de temmps
shannon_indices = df_filtered.groupby('Wits_sample')['workTitle'].apply(shannon_diversity).reset_index()
shannon_indices.columns = ['Wits_sample', 'Shannon_Index']
print(shannon_indices)

# Visualisation du résultat
plt.figure(figsize=(10, 6))
plt.plot(shannon_indices['Wits_sample'], shannon_indices['Shannon_Index'], marker='o', linestyle='-', color='b')
plt.xlabel('Période')
plt.ylabel('Indice de Shannon')
plt.title('Indice de Shannon par période de temps')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.savefig('shannon_indices_by_period.png')
plt.show()
output_file_path = 'shannon_indices_by_period.xlsx'
shannon_indices.to_excel(output_file_path, index=False)

print("Les indices de Shannon ont été calculés, enregistrés et la courbe a été créée avec succès.")
